GAN

resizing all images
CycleGAN typically requires images of a fixed size (e.g., 256x256). Use the following script to resize all images in a directory

In [7]:
from PIL import Image
import os
import shutil

def convert_images(input_dir, output_dir):
    """
    Convert images in the input directory to RGB and save them as JPEG in the output directory.
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    for root, _, files in os.walk(input_dir):
        for file in files:
            file_path = os.path.join(root, file)
            try:
                with Image.open(file_path) as img:
                    # Convert image to RGB if it's not already in that mode
                    if img.mode not in ("RGB", "L"):
                        img = img.convert("RGB")
                    
                    # Create corresponding output path
                    relative_path = os.path.relpath(root, input_dir)
                    save_dir = os.path.join(output_dir, relative_path)
                    os.makedirs(save_dir, exist_ok=True)
                    
                    # Save image as JPEG
                    save_path = os.path.join(save_dir, f"{os.path.splitext(file)[0]}.jpg")
                    img.save(save_path, "JPEG")
            except Exception as e:
                print(f"Error processing {file_path}: {e}")

def move_processed_images(output_dir, original_dir):
    """
    Move processed images back to their respective original folders.
    """
    for root, _, files in os.walk(output_dir):
        for file in files:
            src_path = os.path.join(root, file)
            relative_path = os.path.relpath(root, output_dir)
            dest_dir = os.path.join(original_dir, relative_path)
            dest_path = os.path.join(dest_dir, file)
            
            # Ensure the destination folder exists
            os.makedirs(dest_dir, exist_ok=True)
            
            # Move the file
            shutil.move(src_path, dest_path)

# Set the paths for your dataset
input_directory = "dataset"
output_directory = "processed_dataset"

# Step 1: Convert the images
convert_images(input_directory, output_directory)

# Step 2: Move the processed images back to their respective folders
move_processed_images(output_directory, input_directory)

# Step 3: Clean up the temporary processed folder
shutil.rmtree(output_directory)


In [8]:
from PIL import Image
import os

def resize_images(input_dir, output_dir, size=(256, 256)):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for category in os.listdir(input_dir):
        category_path = os.path.join(input_dir, category)
        output_category_path = os.path.join(output_dir, category)
        if not os.path.exists(output_category_path):
            os.makedirs(output_category_path)

        for filename in os.listdir(category_path):
            if filename.endswith(".jpg") or filename.endswith(".png"):
                img_path = os.path.join(category_path, filename)
                output_path = os.path.join(output_category_path, filename)
                try:
                    img = Image.open(img_path)
                    img = img.resize(size)
                    img.save(output_path)
                except Exception as e:
                    print(f"Error processing {img_path}: {e}")

# Example usage
resize_images("dataset", "processed_dataset")


DataSet Preparation

In [12]:
import os
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

class WeatherDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        
        # Collect all images from subdirectories
        for category in os.listdir(root_dir):
            category_path = os.path.join(root_dir, category)
            if os.path.isdir(category_path):
                for img_name in os.listdir(category_path):
                    self.image_paths.append(os.path.join(category_path, img_name))
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image

# Transform and DataLoader
transform = transforms.Compose([
    transforms.Resize((256, 256)),  # Resize images
    transforms.ToTensor(),          # Convert to tensor
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize to [-1, 1]
])

dataset = WeatherDataset(root_dir="dataset", transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)


Step 3: 

Define GAN Architecture


Basic GAN Components:

Generator (G):
Takes random noise as input and generates images.

Discriminator (D):
Differentiates between real and generated images.

Generator Example:

In [13]:
import torch.nn as nn

class Generator(nn.Module):
    def __init__(self, noise_dim=100):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            nn.Linear(noise_dim, 256),
            nn.ReLU(True),
            nn.Linear(256, 512),
            nn.ReLU(True),
            nn.Linear(512, 1024),
            nn.ReLU(True),
            nn.Linear(1024, 3 * 256 * 256),  # Output image size
            nn.Tanh()  # Normalize output to [-1, 1]
        )
    
    def forward(self, z):
        return self.main(z).view(-1, 3, 256, 256)


Discriminator Example

In [14]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            nn.Linear(3 * 256 * 256, 1024),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(1024, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
            nn.Sigmoid()  # Output probability of real/fake
        )
    
    def forward(self, img):
        return self.main(img.view(img.size(0), -1))


Step 4: Training the GAN
Define Loss and Optimizers:

Binary Cross-Entropy Loss for Discriminator.
Adam optimizer for both Generator and Discriminator.
Training Loop:

Alternate between training G and D.
Training Code

In [15]:
import torch
print(torch.cuda.is_available())  # Should return True if CUDA is working
print(torch.cuda.current_device())  # Should show your GPU ID (if available)


True
0


In [ ]:
import torch
import torch.optim as optim
import torch.nn as nn
from tqdm import tqdm

# Initialize models
G = Generator(noise_dim=100).cuda()
D = Discriminator().cuda()

# Loss and optimizers
criterion = nn.BCELoss()
optimizer_G = optim.Adam(G.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizer_D = optim.Adam(D.parameters(), lr=0.0002, betas=(0.5, 0.999))

# Training
epochs = 100
noise_dim = 100

for epoch in range(epochs):
    for real_imgs in tqdm(dataloader):
        real_imgs = real_imgs.cuda()

        # Train Discriminator
        optimizer_D.zero_grad()
        real_labels = torch.ones(real_imgs.size(0), 1).cuda()
        fake_labels = torch.zeros(real_imgs.size(0), 1).cuda()
        
        real_outputs = D(real_imgs)
        real_loss = criterion(real_outputs, real_labels)
        
        noise = torch.randn(real_imgs.size(0), noise_dim).cuda()
        fake_imgs = G(noise)
        fake_outputs = D(fake_imgs.detach())
        fake_loss = criterion(fake_outputs, fake_labels)
        
        d_loss = real_loss + fake_loss
        d_loss.backward()
        optimizer_D.step()
        
        # Train Generator
        optimizer_G.zero_grad()
        fake_outputs = D(fake_imgs)
        g_loss = criterion(fake_outputs, real_labels)
        g_loss.backward()
        optimizer_G.step()
    
    print(f"Epoch [{epoch+1}/{epochs}] | D Loss: {d_loss.item()} | G Loss: {g_loss.item()}")

    # Save generated images every few epochs
    if (epoch + 1) % 10 == 0:
        with torch.no_grad():
            sample_noise = torch.randn(16, noise_dim).cuda()
            generated_images = G(sample_noise).cpu()
            torchvision.utils.save_image(generated_images, f"generated_epoch_{epoch+1}.png", normalize=True)


100%|████████████████████| 215/215 [22:44<00:00,  6.34s/it]


Epoch [1/100] | D Loss: 0.6313520669937134 | G Loss: 0.7607000470161438


100%|████████████████████| 215/215 [23:48<00:00,  6.65s/it]


Epoch [2/100] | D Loss: 0.7778439521789551 | G Loss: 0.8395625352859497


100%|████████████████████| 215/215 [27:25<00:00,  7.65s/it]


Epoch [3/100] | D Loss: 0.6752406358718872 | G Loss: 1.2444467544555664


  4%|▉                     | 9/215 [01:02<26:37,  7.76s/it]